In [2]:
import pandas as pd

In [3]:
data_raw = pd.read_csv("data/movies_metadata.csv")

c:\users\wahba\pycharmprojects\truelayer challange\venv\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# only keep the needed columns
data = data_raw[["overview","title","genres"]]
data = data[~data['overview'].isna()]
# create a mask indication where a genre value exists
has_genres_mask = data['genres'] != "[]"
genres = data['genres'][has_genres_mask]

In [5]:
"""
AST allows us to evaluate the string list in each genre entry.
basically creates a list from a string with list content.
"""
import ast

def make_labels(strings):
    evaluated_string = ast.literal_eval(strings)
    return [g['name'] for g in evaluated_string]

genres_list = genres.apply(make_labels)

In [6]:
from sklearn.preprocessing import MultiLabelBinarizer
"""
the MultiLabelBinarizer simply looks through all entries in a list and creates list containing unique labels.
"""
labeler = MultiLabelBinarizer()
labeler.fit(genres_list)

MultiLabelBinarizer(classes=None, sparse_output=False)

In [7]:
from joblib import dump

dump(labeler, "model/class_labler.joblib")
print(labeler.classes_)


['Action' 'Adventure' 'Animation' 'Aniplex' 'BROSTA TV'
 'Carousel Productions' 'Comedy' 'Crime' 'Documentary' 'Drama' 'Family'
 'Fantasy' 'Foreign' 'GoHands' 'History' 'Horror'
 'Mardock Scramble Production Committee' 'Music' 'Mystery' 'Odyssey Media'
 'Pulser Productions' 'Rogue State' 'Romance' 'Science Fiction'
 'Sentai Filmworks' 'TV Movie' 'Telescene Film Group Productions'
 'The Cartel' 'Thriller' 'Vision View Entertainment' 'War' 'Western']


In [8]:
# for the next steps its really important to make sure both entries are of string type
pre_X1 = data['title'][has_genres_mask].astype(dtype="str")
pre_X2 = data['overview'][has_genres_mask].astype(dtype="str")

Now the 

In [9]:
"""]
now we have a binary list representing the genres.
this can be used directly in the training
"""
y = labeler.transform(genres_list)

In [10]:
"""
Just checking the size of all entries, Now we are sure everything is aligned correctly
"""
print(len(y))
print(len(pre_X1))
print(len(pre_X2))

42327
42327
42327


In [11]:
from tensorflow import keras
import tensorflow as tf

In [12]:

# The maximum number of words to be used.
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 100 # This is a fixed value, in this case
tokenizer = keras.preprocessing.text.Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(pre_X1 + pre_X2)
word_index = tokenizer.word_index

dump(tokenizer, "model/tokenizer.joblib")


['model/tokenizer.joblib']

In [13]:
import numpy as np
def max_len(array_):
    max_l = 0
    for x in array_:
        if len(x) > max_l:
            max_l = len(x)
    return max_l
X1 = tokenizer.texts_to_sequences(pre_X1)
X2 = tokenizer.texts_to_sequences(pre_X2)

X1_max_len = max_len(X1)
X2_max_len = max_len(X2)

X1 = keras.preprocessing.sequence.pad_sequences(X1, maxlen=X1_max_len)
X2 = keras.preprocessing.sequence.pad_sequences(X2, maxlen=X2_max_len)

In [14]:
"""
this section attempted to split the data into training and validation sets
However, this was scrapped in favor of letting the fit function
deal with the validation split
"""
# from sklearn.model_selection import train_test_split
# 
# X1_train, X1_test = train_test_split(X1, shuffle = False)
# X2_train, X2_test = train_test_split(X2, shuffle = False)
# Y_train, Y_test = train_test_split(y, shuffle = False)

# print(X1_train.shape,Y_train.shape)
# print(X1_test.shape,Y_test.shape)

'\nthis section attempted to split the data into training and validation sets\nHowever, this was scrapped in favor of letting the fit function\ndeal with the validation split\n'

In [15]:
"""
load the classifier we constructed
"""
from model import GenreClassifier

model = GenreClassifier(len(labeler.classes_), MAX_NB_WORDS, X1_max_len, X2_max_len)


Using TensorFlow backend.
W0704 12:55:40.418211 15964 deprecation_wrapper.py:119] From c:\users\wahba\pycharmprojects\truelayer challange\venv\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0704 12:55:40.432175 15964 deprecation_wrapper.py:119] From c:\users\wahba\pycharmprojects\truelayer challange\venv\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0704 12:55:40.434170 15964 deprecation_wrapper.py:119] From c:\users\wahba\pycharmprojects\truelayer challange\venv\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0704 12:55:41.178181 15964 deprecation_wrapper.py:119] From c:\users\wahba\pycharmprojects\truelayer challange\venv\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optim

In [ ]:
"""
train the classifier
the specified batch size requires lots of memory, consider reducing the value on first run
"""
EPOCHS = 33
BATCH_SIZE = 1024
history = model.fit(X1, X2, y,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_split=0.2,
                    shuffle=True)

W0704 12:55:41.391610 15964 deprecation.py:323] From c:\users\wahba\pycharmprojects\truelayer challange\venv\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0704 12:55:43.584745 15964 deprecation_wrapper.py:119] From c:\users\wahba\pycharmprojects\truelayer challange\venv\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 33861 samples, validate on 8466 samples
Epoch 1/33
33861/33861 [==============================] - 308s 9ms/step - loss: 5.9980 - acc: 0.2644 - val_loss: 5.2836 - val_acc: 0.2566
Epoch 2/33
33861/33861 [==============================] - 524s 15ms/step - loss: 5.6962 - acc: 0.2729 - val_loss: 5.0792 - val_acc: 0.2566
Epoch 3/33
19456/33861 [================>.............] - ETA: 4:29 - loss: 5.5742 - acc: 0.2726

In [ ]:
model.save("model/simple_text_classifier_33.h5")

In [49]:
from matplotlib import pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: name 'history' is not defined